# _Vehicle Tracker_
This application reads a video of a highway traffic, defines a region of interest and tracks objects in drastic movement (vehicles, in this case) while also assigning them an ID, according to the order in which they appear. I made this mainly with the intention of taking my first steps with Computer Vision and OpenCV.

### Imports
The only ones needed are _hypot_, a function from the _math_ module that computes the multidimensional euclidean distance from the origin to a point, and _cv2_, the module import name for _opencv-python_.

In [1]:
from math import hypot
import cv2

### Centroid Tracker
In the used video, multiple objects are often detected in consecutive frames and need to be associated across frames. Therefore, for consistent tracking of objects over time, we can create a class that, using the centroids of the bounding boxes, determines whether an object in the current frame corresponds to an existing tracked object or is a new object.

In [2]:
class CentroidTracker:
    def __init__(self):
        self.centroids = {}
        self.id_count = 0

    def update(self, boxes, threshold):
        box_ids = []

        for x, y, w, h in boxes:
            c_x = (2 * x + w) // 2
            c_y = (2 * y + h) // 2

            same_object = False
            for id, pt in self.centroids.items():
                dist = hypot(c_x - pt[0], c_y - pt[1])
                if dist < threshold:
                    self.centroids[id] = (c_x, c_y)
                    box_ids.append([x, y, w, h, id])
                    same_object = True
                    break

            if same_object is False:
                self.centroids[self.id_count] = (c_x, c_y)
                box_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        new_centroids = {}
        for _, _, _, _, id in box_ids:
            new_centroids[id] = self.centroids[id]

        self.centroids = new_centroids.copy()

        return box_ids


### Pre-processing
These 3 global variables are declared outside the main loop to then be used within it:
- `capture`: reads the frames of `traffic.mp4` during the subsequent processing.
- `detector`: executes the _Gaussian Mixture-based Background/Foreground Segmentation_ algorithm.
- `tracker`: associates bounding boxes and unique IDs to objects detected in the subsequently processed video frames.

In [3]:
capture = cv2.VideoCapture("traffic.mp4")
detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=60)
tracker = CentroidTracker()

### Main Loop
The operation of the application consists of the following steps:
1. Reading frames from a video
2. Defining a region of interest (ROI)
3. Performing background subtraction and object detection within the defined ROI
4. Associating detected objects with existing tracked objects
5. Visualizing the results by displaying the processed frames with bounding boxes and IDs

In [4]:
while True:
    # Step 1
    _, frame = capture.read()

    # Step 2
    roi = frame[260:, :660]

    # Step 3
    detections = []
    mask = detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # Step 4
    box_ids = tracker.update(detections, 25)
    for x, y, w, h, id in box_ids:
        cv2.putText(
            roi,
            f"Vehicle #{id}",
            (x, y - 15),
            cv2.FONT_HERSHEY_PLAIN,
            1,
            (0, 255, 0),
            2,
        )
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 0, 255), 3)

    # Step 5
    cv2.imshow("ROI", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

capture.release()
cv2.destroyAllWindows()
